In [1]:
import arcpy, os
import arcpy
from osgeo import gdal

arcpy.env.addOutputsToMap = True   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True

In [2]:
# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

arcpy.env.workspace = directory

In [5]:
toolbox_path = r"C:\Program Files\ETSpatialTechniques\ETGeoWizards\ETGeoWizards12_Pro.pyt"
tool_name = "ET_GW"

# Execute the tool from the toolbox
arcpy.ImportToolbox(toolbox_path)
#arcpy.ETGeoWizards.SplitPolylinesWithLayer('Primaire_watergangen', 'Network_n_filter', 'Primaire_watergangen_split', "1")


<module 'ETGeoWizards'>

In [3]:
PW = "Primaire_Watergangen"
RS = "RchSegments"

In [29]:
arcpy.ETGeoWizards.SnapPolylines(RS, PW, "RchSegments_snap" , "5", "Node", "Vertex")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\RchSegments_snap.shp'>

In [22]:
arcpy.ETGeoWizards.ExportNodes("RchSegments_snap", "RchSegments_snap_nodes", "True", "True", "True")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\RchSegments_snap_nodes.shp'>

In [28]:
arcpy.ETGeoWizards.SnapPoints("RchSegments_snap_nodes", "Primaire_watergangen", "Primaire_watergangen_Nodes_snap", "8" , "Nearest")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\Primaire_watergangen_Nodes_snap_2.shp'>

In [30]:
arcpy.ETGeoWizards.SplitPolylinesWithLayer("Primaire_watergangen", "Primaire_watergangen_Nodes_snap", "Primaire_watergangen_split", "8")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\Primaire_watergangen_split.shp'>

In [36]:
# Set the input shapefile
input_shapefile = "Primaire_watergangen_split"

# Set the name of the new field
new_field_name = "ID_point"

# Add the new field to the shapefile
arcpy.AddField_management(input_shapefile, new_field_name, "LONG")

# Create an update cursor to populate the new field with sequential numbers
with arcpy.da.UpdateCursor(input_shapefile, new_field_name) as cursor:
    counter = 1
    for row in cursor:
        row[0] = counter
        counter += 1
        cursor.updateRow(row)

print("Sequential numbers added to the field.")


Sequential numbers added to the field.


In [43]:
import arcpy

def spatial_join():
    Primaire_watergangen_split = "Primaire_watergangen_split"
    Primaire_watergangen_Nodes_snap = "Primaire_watergangen_Nodes_snap"

    Primaire_watergangen_split_S_shp = "TEST"

    field_mapping = """
    CODE "CODE" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,CODE,0,80;
    NAAM "NAAM" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,NAAM,0,80;
    STATUSOBJE "STATUSOBJE" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,STATUSOBJE,-1,-1;
    SOORTOPPWA "SOORTOPPWA" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,SOORTOPPWA,-1,-1;
    CATEGORIEO "CATEGORIEO" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,CATEGORIEO,-1,-1;
    PERSISTENT "PERSISTENT" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,PERSISTENT,-1,-1;
    WS_OND_DRO "WS_OND_DRO" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_OND_DRO,-1,-1;
    WS_ONDH_NA "WS_ONDH_NA" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_ONDH_NA,-1,-1;
    WS_ONDH_BG "WS_ONDH_BG" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_ONDH_BG,-1,-1;
    WS_ZANDSLO "WS_ZANDSLO" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,WS_ZANDSLO,0,80;
    DOSSIERMAP "DOSSIERMAP" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,DOSSIERMAP,0,80;
    IWS_W_WATP "IWS_W_WATP" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATP,-1,-1;
    IWS_W_WATB "IWS_W_WATB" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATB,-1,-1;
    IWS_W_WATD "IWS_W_WATD" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATD,-1,-1;
    IWS_W_INST "IWS_W_INST" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_INST,-1,-1;
    IWS_W_PLAS "IWS_W_PLAS" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_PLAS,-1,-1;
    IWS_W_BRPL "IWS_W_BRPL" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BRPL,-1,-1;
    IWS_W_TALU "IWS_W_TALU" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_TALU,-1,-1;
    IWS_W_BODB "IWS_W_BODB" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BODB,-1,-1;
    IWS_W_BODH "IWS_W_BODH" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BODH,-1,-1;
    WS_ONDERHO "WS_ONDERHO" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,WS_ONDERHO,-1,-1;
    NAAMSPACE "NAAMSPACE" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,NAAMSPACE,0,80;
    LENGTE "LENGTE" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,LENGTE,-1,-1;
    ET_Type "ET_Type" true true false 80 Text 0 0,First,#,Primaire_watergangen_Nodes_snap,ET_Type,0,80;
    ET_Valency "ET_Valency" true true false 9 Long 0 9,First,#,Primaire_watergangen_Nodes_snap,ET_Valency,-1,-1;
    ET_Status "ET_Status" true true false 80 Text 0 0,First,#,Primaire_watergangen_Nodes_snap,ET_Status,0,80;
    MIN_ID "MIN_ID" true true false 255 Long 0 0,Min,#,Primaire_watergangen_Nodes_snap,ID_point,-1,-1;
    MAX_ID "MAX_ID" true true false 255 Long 0 0,Max,#,Primaire_watergangen_Nodes_snap,ID_point,-1,-1
    """

    arcpy.analysis.SpatialJoin(target_features=Primaire_watergangen_split,
                               join_features=Primaire_watergangen_Nodes_snap,
                               out_feature_class=Primaire_watergangen_split_S_shp,
                               join_operation="JOIN_ONE_TO_ONE",
                               join_type="KEEP_ALL",
                               field_mapping=field_mapping,
                               match_option="INTERSECT",
                               search_radius="",
                               distance_field_name="")
spatial_join()

In [45]:

    Primaire_watergangen_split_S_shp = "H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\Primaire_watergangen_split_S.shp"
    arcpy.analysis.SpatialJoin(target_features="Primaire_watergangen_split", join_features="Primaire_watergangen_Nodes_snap", out_feature_class="TESTT2", join_operation="JOIN_ONE_TO_ONE", join_type="KEEP_ALL", field_mapping="CODE \"CODE\" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,CODE,0,80;NAAM \"NAAM\" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,NAAM,0,80;STATUSOBJE \"STATUSOBJE\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,STATUSOBJE,-1,-1;SOORTOPPWA \"SOORTOPPWA\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,SOORTOPPWA,-1,-1;CATEGORIEO \"CATEGORIEO\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,CATEGORIEO,-1,-1;PERSISTENT \"PERSISTENT\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,PERSISTENT,-1,-1;WS_OND_DRO \"WS_OND_DRO\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_OND_DRO,-1,-1;WS_ONDH_NA \"WS_ONDH_NA\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_ONDH_NA,-1,-1;WS_ONDH_BG \"WS_ONDH_BG\" true true false 9 Long 0 9,First,#,Primaire_watergangen_split,WS_ONDH_BG,-1,-1;WS_ZANDSLO \"WS_ZANDSLO\" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,WS_ZANDSLO,0,80;DOSSIERMAP \"DOSSIERMAP\" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,DOSSIERMAP,0,80;IWS_W_WATP \"IWS_W_WATP\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATP,-1,-1;IWS_W_WATB \"IWS_W_WATB\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATB,-1,-1;IWS_W_WATD \"IWS_W_WATD\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_WATD,-1,-1;IWS_W_INST \"IWS_W_INST\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_INST,-1,-1;IWS_W_PLAS \"IWS_W_PLAS\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_PLAS,-1,-1;IWS_W_BRPL \"IWS_W_BRPL\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BRPL,-1,-1;IWS_W_TALU \"IWS_W_TALU\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_TALU,-1,-1;IWS_W_BODB \"IWS_W_BODB\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BODB,-1,-1;IWS_W_BODH \"IWS_W_BODH\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,IWS_W_BODH,-1,-1;WS_ONDERHO \"WS_ONDERHO\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,WS_ONDERHO,-1,-1;NAAMSPACE \"NAAMSPACE\" true true false 80 Text 0 0,First,#,Primaire_watergangen_split,NAAMSPACE,0,80;LENGTE \"LENGTE\" true true false 24 Double 15 23,First,#,Primaire_watergangen_split,LENGTE,-1,-1;ET_Type \"ET_Type\" true true false 80 Text 0 0,First,#,Primaire_watergangen_Nodes_snap,ET_Type,0,80;ET_Valency \"ET_Valency\" true true false 9 Long 0 9,First,#,Primaire_watergangen_Nodes_snap,ET_Valency,-1,-1;ET_Status \"ET_Status\" true true false 80 Text 0 0,First,#,Primaire_watergangen_Nodes_snap,ET_Status,0,80;MIN_ID \"MIN_ID\" true true false 255 Long 0 0,Min,#,Primaire_watergangen_Nodes_snap,ID_point,-1,-1;MAX_ID \"MAX_ID\" true true false 255 Long 0 0,Max,#,Primaire_watergangen_Nodes_snap,ID_point,-1,-1", match_option="INTERSECT", search_radius="", distance_field_name="")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\TESTT2.shp'>

In [40]:
import arcpy

def add_id_fields(point_layer, line_layer, output_feature_class):
    # Perform the spatial join
    arcpy.SpatialJoin_analysis(target_features=line_layer,
                               join_features=point_layer,
                               out_feature_class=output_feature_class,
                               join_operation="JOIN_ONE_TO_MANY",
                               join_type="KEEP_COMMON",
                               match_option="INTERSECT")

    # Add fields to the output feature class
    arcpy.AddField_management(output_feature_class, "Min_id", "LONG")
    arcpy.AddField_management(output_feature_class, "Max_id", "LONG")

    # Calculate the maximum values from the point layer fields
    arcpy.CalculateField_management(output_feature_class, "Min_id", "min(!ID_point!)", "PYTHON")
    arcpy.CalculateField_management(output_feature_class, "Max_id", "max(!ID_point!)", "PYTHON")

    print("Spatial join and field calculations completed.")

# Call the function
add_id_fields(os.path.join(directory,"Primaire_watergangen_Nodes_snap.shp"), os.path.join(directory,"Primaire_watergangen_split.shp"), os.path.join(directory,"Primaire_watergangen_split_id.shp"))


Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: 'int' object is not iterable


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CalculateField).


# Archief

In [25]:
import arcpy

point_fc = "Primaire_watergangen_Nodes_snap"
line_shapefile = "Primaire_watergangen"

arcpy.MakeFeatureLayer_management(line_shapefile, "line_layer")
arcpy.SelectLayerByLocation_management(point_fc, "INTERSECT", "line_layer", "", "NEW_SELECTION")
selected_count = arcpy.GetCount_management(point_fc)

arcpy.MakeFeatureLayer_management(point_fc, "point_layer")
arcpy.SelectLayerByLocation_management("point_layer", "INTERSECT", "line_layer", "", "SWITCH_SELECTION")
non_intersect_count = arcpy.GetCount_management("point_layer")

print(f"Selected points: {selected_count}")
print(f"Non-intersecting points: {non_intersect_count}")


Selected points: 18451
Non-intersecting points: 1044


In [27]:
arcpy.ETGeoWizards.SnapPoints("RchSegments_ExportNodes", "Primaire_watergangen", "RchSegments_ExportNodes_snap_2", "9" , "Nearest")

ExecuteError: An error 1 ocurred! See ETGeoWizards.log file for details.
Failed to execute (SnapPoints).


Analyzes the nodes of a polyline layer and exports them as a point layer.

In [19]:
arcpy.ETGeoWizards.ExportNodes("Primaire_watergangen", "RchSegments_nodes", "Export Dangling" , "Export Pseudo", "Export Regular")

ExecuteError: ERROR 000622: Failed to execute (Export Nodes). Parameters are not valid.
ERROR 000628: Cannot set input into parameter dangles.


To select all line features in a shapefile that touch another line shapefile without generating new layers in arcpy, you can use the SelectLayerByLocation_management tool. Here's an example code snippet:

In [13]:
import arcpy

# Specify the paths to your line shapefiles
line_shapefile1 = "RchSegments_snap_15"
line_shapefile2 = "RchSegments_snap_15_Original"

# Create feature layers
arcpy.MakeFeatureLayer_management(line_shapefile1, "line_layer1")
arcpy.MakeFeatureLayer_management(line_shapefile2, "line_layer2")

# Select line features that touch with another line shapefile
arcpy.SelectLayerByLocation_management("line_layer1", "BOUNDARY_TOUCHES", "line_layer2", "", "NEW_SELECTION")

# Iterate over the selected features and print their attributes
with arcpy.da.SearchCursor("line_layer1", "*") as cursor:
    for row in cursor:
        print(row)


(131, (134785.3038019764, 446426.40876987507), 'H070281_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'HR_9', 'fxcpH070281_3', 'Original', 8360)
(132, (134799.68666970375, 446498.3586050611), 'H070281_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH070281_3', 'fxcpH070281_4', 'Original', 8360)
(133, (134812.84216772733, 446566.49423518707), 'H070281_s2', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH070281_4', 'eH070281', 'Original', 8360)
(225, (135453.9639, 449972.5096000005), 'H074154_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH074154', 'eH074154', 'Original', 8360)
(421, (134289.62940350088, 449940.4985175935), 'H060869_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH060869', 'fxcpH060869_3', 'Snapped', 8360)
(450, (130243.207900001, 451608.86475), 'H063398_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH063398', 'eH063398', 'Original', 8360)
(451, (130192.59325000101, 451525.94589999894), 'H063399_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH063399', 'sH06

(1458, (127421.0835499985, 457779.93675), 'H069999_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH069618', 'sH070011', 'Original', 8360)
(1459, (127698.118799999, 457885.27514999954), 'H070004_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH070004', 'eH070004', 'Original', 8360)
(1460, (127666.58883766441, 457857.2325370146), 'H070005_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH062247', 'fxcpH070005_4', 'Original', 8360)
(1461, (127693.65013766443, 457881.4874870146), 'H070005_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH070005_4', 'sH070004', 'Original', 8360)
(1462, (127701.97779999852, 457887.9975000006), 'H070007_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH070004', 'sH070008', 'Original', 8360)
(1513, (132313.797900001, 454726.04149999854), 'H062121_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH062121', 'sH062180', 'Original', 8360)
(1514, (126216.15279956051, 462307.46445328393), 'H071246_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH071246', 'fxc

(4646, (118852.40440000051, 456286.86289999797), 'H025769_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH025394', 'eH025769', 'Original', 8360)
(4647, (111464.03099948884, 455817.06109856616), 'H022913_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH022913', 'eH028215', 'Snapped', 8360)
(4648, (124298.90294999996, 457149.4713000005), 'H032573_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH032573', 'eH032573', 'Snapped', 8360)
(4650, (124033.315649999, 457245.97750000097), 'H032933_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH032904', 'eH032932', 'Original', 8360)
(4651, (123701.23087028092, 456546.76426229294), 'H027963_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH027963', 'fxcpH027963_4', 'Original', 8360)
(4652, (123697.61372028192, 456500.0856622925), 'H027963_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH027963_4', 'eH027963', 'Original', 8360)
(4653, (123695.24215000149, 456465.03964999854), 'H027964_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH02796

(6285, (118346.84055000001, 443309.6000500005), 'H043211_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH043211', 'eH043211', 'Original', 8360)
(6334, (118785.19724999955, 440100.25355000043), 'H042266_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH042266', 'sH074515', 'Snapped', 8360)
(6336, (119552.55324373709, 440890.3948159779), 'H040260_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH042521', 'fxcpH040260_3', 'Original', 8360)
(6337, (119638.03249373611, 440938.57726597844), 'H040260_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH040260_3', 'eH040260', 'Original', 8360)
(6338, (117885.52649999951, 441549.9054499995), 'H042565_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH043085', 'sH043087', 'Original', 8360)
(6339, (123170.06109396603, 442516.03066115046), 'H046753_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH046753', 'fxcpH046753_4', 'Original', 8360)
(6340, (123156.56539396654, 442510.9258611499), 'H046753_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', '

(6396, (119156.03862306048, 443691.65743762755), 'H049305_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH049663', 'fxcpH049305_5', 'Original', 8360)
(6397, (119126.45227927904, 443674.033166217), 'H049305_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH049305_5', 'fxcpH049305_6', 'Original', 8360)
(6398, (119049.10461241551, 443632.87931096903), 'H049305_s2', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH049305_6', 'fxcpH049305_7', 'Original', 8360)
(6399, (118948.77570619545, 443581.15378237964), 'H049305_s3', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH049305_7', 'eH049305', 'Original', 8360)
(6400, (120430.83549138301, 444467.56853607844), 'H044921_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH044921', 'fxcpH044921_3', 'Original', 8360)
(6401, (120354.1855913825, 444420.87933607947), 'H044921_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH044921_3', 'sH049714', 'Original', 8360)
(6402, (120478.27805, 444495.41275), 'H045138_1', ' ', 'Flow - Channel', 'SBK

...

(8420, (117756.7130500005, 441747.8322000005), 'H074834_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH074834', 'sH043085', 'Original', 8360)
(8421, (117731.07815000051, 441785.7311000005), 'H074833_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH046415', 'sH074834', 'Original', 8360)
(8427, (131580.3955000005, 459433.4631999996), 'H061878_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH068630', 'sH061879', 'Original', 8360)
(8441, (127935.6301000005, 457725.0358499995), 'H075145_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH075145', 'sH069945', 'Original', 8360)
(8442, (127853.304199999, 457772.5845499985), 'H075146_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH070008', 'sH075145', 'Original', 8360)
(8491, (119308.14450000048, 442499.3500000015), 'H042881_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH046719', 'sH046756', 'Original', 8360)
(8494, (134220.551349999, 448122.75175), 'H074947_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH055635', 'eH065038', 'Origina

(11765, (117668.9154467324, 458727.90007662005), 'H036316_1_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH028518', 'fxcpH036316_1_1', 'Original', 8360)
(11766, (117833.87602030781, 458912.53333864815), 'H036316_1_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH036316_1_1', 'fxcpH036316_1_2', 'Original', 8360)
(11767, (117998.5165139514, 459097.45699630113), 'H036316_1_s2', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH036316_1_2', 'fxcpH036316_1_3', 'Original', 8360)
(11768, (118161.77791689846, 459283.6177118205), 'H036316_1_s3', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH036316_1_3', 'fxcpH036316_1_4', 'Original', 8360)
(11769, (118325.52895786049, 459469.37358068), 'H036316_1_s4', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH036316_1_4', 'H036316', 'Original', 8360)
(11829, (136153.123870486, 453098.340147924), 'H057955_2_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'H057955', 'fxcpH057955_2_1', 'Original', 8360)
(11830, (136187.08240238746, 452902.651149503

(14143, (135057.73255973315, 448870.6862987457), 'H055874d_2_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH055874d_2_1', 'eH055874', 'Original', 8360)
(14148, (135109.87913260545, 450223.89486054867), 'H059793_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH059793', 'fxcpH059793_1', 'Snapped', 8360)
(14149, (135039.61298563913, 450080.1093875964), 'H059793_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH059793_1', 'KP_PG0049-32', 'Original', 8360)
(14150, (135001.44584565802, 450002.0080241295), 'H059793_3_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'KP_PG0049-32', 'H059793', 'Original', 8360)
(14151, (135551.7529213905, 451065.3549143625), 'H055895d_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH074523', 'KP_PG0049-33', 'Original', 8360)
(14152, (135494.95521609648, 450957.6829926435), 'H055895d_3_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'KP_PG0049-33', 'H055895d', 'Original', 8360)
(14153, (135663.18747338367, 451083.42218841694), 'H055856_1', ' ', 'Flow -

(15942, (128189.83406615168, 458761.5185773615), 'H067114_2_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH067114_2_4', 'fxcpH067114_2_5', 'Original', 8360)
(15943, (128200.8953202565, 458793.8895914935), 'H067114_2_s2', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH067114_2_5', 'fxcpH067114_2_6', 'Original', 8360)
(15944, (128220.35397554838, 458851.99405669555), 'H067114_2_s3', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH067114_2_6', 'sH074872', 'Original', 8360)
(15945, (128842.58644430674, 459306.05140293896), 'H068266_1_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH061877', 'KP_PG0343-1', 'Snapped', 8360)
(15946, (128802.15319430799, 459321.2678029405), 'H068266_2_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'KP_PG0343-1', 'eH068266', 'Original', 8360)
(15963, (127471.65375191352, 457801.93722374155), 'H070012_1_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH070012', 'KP_PG0349-2', 'Original', 8360)
(15964, (127432.71410191449, 457887.04112374195), 'H070012

(18855, (117486.56780580251, 458525.7029185249), 'H028518_1_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'eH028517', 'KP_PG1383-2', 'Original', 8360)
(18856, (117536.89974871576, 458580.9098656582), 'H028518_2_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'KP_PG1383-2', 'fxcpH028518_2_3', 'Original', 8360)
(18857, (117583.60394291328, 458631.4124971332), 'H028518_2_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH028518_2_3', 'eH028518', 'Original', 8360)
(18858, (116425.74858378191, 460574.9035038389), 'H037110_1_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'sH037110', 'fxcpH037110_1_1', 'Original', 8360)
(18859, (116337.09537222743, 460670.2219000429), 'H037110_1_s1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'fxcpH037110_1_1', 'KP_PG1403-1', 'Original', 8360)
(18860, (116218.5993884445, 460798.976846204), 'H037110_2_1', ' ', 'Flow - Channel', 'SBK_CHANNEL', ' ', 'KP_PG1403-1', 'eH037110', 'Original', 8360)
(18867, (141075.3130726885, 455464.84228928393), 'H014628_1_1', ' 

In [7]:
arcpy.env.addOutputsToMap = True
arcpy.ETGeoWizards.SnapPoints("Network_n_filter_no_intercect_BR", "Primaire_watergangen", "Network_n_filter_no_intercect_BR_snap_1", "5" , "Vertex")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\Network_n_filter_no_intercect_BR_snap_1.shp'>

In [25]:
arcpy.env.addOutputsToMap = True
arcpy.ETGeoWizards.SnapPolylines('RchSegments', 'Primaire_watergangen', "RchSegments_snap_20" , "5", "Node", "Vertex")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\RchSegments_snap_20.shp'>

In [9]:
arcpy.ETGeoWizards.CleanPseudoNodes("RchSegments", "RchSegments_Clean")

<Result 'H:\\DATA\\Service\\Wietse\\Hydraulische toetsing\\HT_Python\\RchSegments_Clean.shp'>

New heading